# Name Change
## Aim

The aim of this pipeline is to change the header of gene expression file based on a reference file that are made externally. 

An example of the pipeline running is shown below

In [81]:
SOS run /Users/haosun/Documents/WG_Reasearch_Assisstant/GIT/twas-dev/Workflow/SOS_change_patient_name.ipynb \
    --ref "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/new_gene_keys_plink.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/" 

INFO: Running Processing_1: change the name of gene_exp
INFO: Processing_1 (index=0) is ignored due to saved signature
INFO: Processing_1 output:   /Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj_rename.txt /Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj_discardID.txt
INFO: Running Processing_2: add position to gene_ID based on ensembl

载入程辑包：‘dplyr’

The following object is masked from ‘package:biomaRt’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Parsed with column specification:
cols(
  .default = col_double(),
  gene_ID = col_character()
)
See spec(...) for full column specifications.
INFO: Processing_2 is completed.
INFO: Processing_2 output:   /Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneC

In [71]:
pwd

/Users/haosun/Documents/WG_Reasearch_Assisstant/GIT/twas-dev/WIP/pipeline_prototype



In [73]:
[global]
# This pipeline changes the patient names of a gene_exp file based on a index reference file 
parameter: ref = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/new_gene_keys_plink.txt"
# A table contaiing the patient ID in the gene_exp file and the patient ID in the plink file
parameter: name_col = 2
# Column position that specify the header of gene_exp in the index
parameter: real_name_col = 1
# Column position that specify the plink patient ID in the index
parameter: gene_exp = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj.txt"
# the patient ID to be changed
parameter: start_at = 2
# the column where the patient ID start

parameter: output = "/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/"
# where to store the output file

# change the name of gene_exp
[Processing_1]
input: ref, gene_exp
output: f'{output}/{_input[1]:bn}_rename.txt',
        f'{output}/{_input[1]:bn}_discardID.txt'

R:  expand= "$[ ]"
    library(dplyr)
    gene_exp = readr::read_delim("$[_input[1]]",delim = " ")
    index = readr::read_delim("$[_input[0]]",delim = "\t")
    name = data.frame(Name = index%>%pull(colnames(index)[$[name_col]]),Real_name = index%>%pull(colnames(index)[$[real_name_col]]))
    df = gene_exp[,$[start_at]:ncol(gene_exp)]
    pos = match(colnames(df), name$Name%>%as.character())%>%na.omit()
    df = df %>%
      select_at(as.character(name$Name[pos]), 
                ~as.character((name$Real_name[pos])))
    final = cbind(gene_exp[,1:$[start_at]-1],df)
    final%>%readr::write_tsv( path = "$[_output[0]]", na = "NA", append = FALSE, col_names = TRUE, quote_escape = "double")
    name$Name[attr(pos,"na.action")]%>%cat(file = "$[_output[1]]", sep = " ")

# add position to gene_ID based on ensembl 
[Processing_2]
input: group_by = 2
output: f'{output}/{_input[0]:bn}_anno.txt'

R:  expand= "$[ ]"
    library("biomaRt")
    library(dplyr)
    gene_exp = readr::read_delim("$[_input[0]]",delim = "\t")
    ensembl = useDataset("hsapiens_gene_ensembl",mart=useMart("ENSEMBL_MART_ENSEMBL"))
    ensembl_df <- getBM(attributes=c("ensembl_gene_id","chromosome_name", "start_position", "end_position"),mart=ensembl)
    my_genes = gene_exp$gene_ID
    my_genes_ann = ensembl_df[match(my_genes, ensembl_df$ensembl_gene_id),]%>%dplyr::select("gene_ID" = ensembl_gene_id, "chr" = chromosome_name, start_position, end_position)
    final = inner_join(my_genes_ann, gene_exp, "gene_ID")
    final%>%readr::write_tsv( path = "$[_output]", na = "NA", append = FALSE, col_names = TRUE, quote_escape = "double")

In [44]:
library("biomaRt")
listMarts()

gene_exp = readr::read_delim("/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/gene_exp/geneCountsResidualsPlusBaselineAgeGenderAdj_rename.txt",delim = "\t")
ensembl = useMart("ENSEMBL_MART_ENSEMBL")
ensembl = useDataset("hsapiens_gene_ensembl",mart=ensembl)
listAttributes(ensembl)[1:100,]


ensembl_df <- getBM(attributes=c("ensembl_gene_id", "ensembl_transcript_id",
                                 "hgnc_symbol","chromosome_name", "start_position", "end_position"),mart=ensembl)
my_genes = gene_exp$gene_ID
my_genes_ann = ensembl_df[match(my_genes, ensembl_df$ensembl_gene_id),]%>%dplyr::select("gene_ID" = ensembl_gene_id, "chr" = chromosome_name, start_position, end_position)
final = inner_join(my_genes_ann, gene_exp, "gene_ID")%>%as.tibble()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Parsed with column specification:
cols(
  .default = col_double(),
  gene_ID = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  geno_samples = col_character(),
  projid = col_double()
)



[1] "20275399" "10442701" "20152393" "20626558" "15176592" "21174691"
   [7] "15146050" "10606903" "20969088" "11319389" "21185180" "10433070"
  [13] "10439580" "10490993" "20266118" "11444465" "21001933" "20984415"
  [19] "20706215" "21112011" "20297791" "21411297" "20550027" "11342432"
  [25] "10481686" "15179365" "20998065" "20327084" "11395417" "10478041"
  [31] "10277308" "20993308" "21117192" "20996621" "20730959" "21405627"
  [37] "20966477" "20594669" "20611993" "10533083" "10460587" "20500815"
  [43] "21142003" "20946545" "10460299" "20956605" "20521752" "20339740"
  [49] "21181988" "11259716" "20128523" "20177982" "10572120" "21174303"
  [55] "21001807" "10669174" "20102505" "20487369" "10479920" "11430815"
  [61] "11310844" "10208143" "10536568" "11697592" "15121461" "20603141"
  [67] "21172121" "20174533" "15113169" "20956867" "21117516" "20610726"
  [73] "10850171" "21412626" "20431359" "20665307" "20376029" "15420223"
  [79] "21401399" "10235948" "15114174" "10684501" "10417638" "20288196"
  [85] "20240514" "20171508" "20969114" "10249336" "20970441" "21000504"
  [91] "11327005" "21193068" "10101291" "20929774" "20660226" "10485914"
  [97] "11349119" "15797073" "20233939" "11460357" "20156469" "15483818"
 [103] "20311676" "15144878" "11467746" "10502798" "20906493" "20164059"
 [109] "10202345" "20236838" "20602974" "20141228" "20139850" "20195344"
 [115] "20147026" "20506325" "21402016" "21145876" "21000766" "20249897"
 [121] "21101658" "10929637" "10290427" "20299973" "11345331" "11157783"
 [127] "20492186" "21411323" "21402304" "15110134" "15115927" "21126085"
 [133] "10976490" "20115276" "20904509" "21191200" "21246218" "20584923"
 [139] "20774339" "20559562" "15139925" "20557128" "20561642" "20532115"
 [145] "20530195" "21232244" "20232186" "11302830" "11072071" "20106383"
 [151] "20234818" "61215831" "20925446" "20219931" "21140119" "20909680"
 [157] "20197364" "20105242" "20130577" "11344452" "21403733" "20990085"
 [163] "21276174" "15147641" "20494368" "10308444" "20225051" "20151388"
 [169] "15702926" "20474724" "10510090" "21189544" "21144735" "21131352"
 [175] "21002524" "21155350" "10986136" "20270768" "10425642" "11200645"
 [181] "21403157" "20390027" "10490579" "11069724" "20630946" "11335695"
 [187] "21274866" "21176585" "21002812" "10275964" "21000630" "20683921"
 [193] "20133764" "10255456" "20416210" "20726273" "10660639" "20224046"
 [199] "11448667" "10499302" "21236608" "11165535" "10291856" "20865035"
 [205] "21120287" "50197261" "50104684" "50107871" "34726040" "31037310"
 [211] "50101109" "51826377" "61344957" "50305165" "50105725" "90447310"
 [217] "50100482" "85666580" "90214403" "50107583" "50108174" "89001223"
 [223] "50300408" "22813029" "50401976" "70883578" "62483327" "94722642"
 [229] "84642424" "70726480" "23870000" "93462021" "43074402" "50302978"
 [235] "50102240" "29933130" "77143621" "15218541" "50401264" "50100518"
 [241] "92113025" "50103967" "25300551" "82684406" "61142759" "95116046"
 [247] "50409406" "22868024" "50101659" "56696733" "50100770" "78072753"
 [253] "50104134" "50104972" "36492755" "50302004" "50404299" "89164957"
 [259] "50109215" "50107457" "12365619" "7883803"  "51815338" "95330358"
 [265] "50410319" "9391376"  "83034844" "72188804" "50100932" "9650662" 
 [271] "50102952" "50302716" "34999847" "50102376" "8109170"  "7253015" 
 [277] "50103381" "5498462"  "31509843" "50404749" "50409370" "50408815"
 [283] "50301251" "66924745" "50302554" "67216828" "74078795" "50407936"
 [289] "54458919" "56102646" "50102088" "34962204" "35941263" "50406057"
 [295] "45566083" "91018909" "50410283" "87779516" "61029627" "53389845"
 [301] "50400709" "50301413" "50301387" "50108886" "3713990"  "82317494"
 [307] "4879843"  "50109189" "37178462" "17929065" "76008327" "37452211"
 [313] "25580771" "46369736" "64216968" "92629514" "50402017" "60848460"
 [319] "50300084" "92996364" "1243685"  "26498416" "70625336" "72650337"
 [325] "80187972" "81852640" "53772202" "9841821"  "482428"  